This file should help make everything run.

NOTE: All the stuff you need to run is in the Modified_OptPolicy File. In 1. Tradeoff Curves/2.Scripts

Sometimes Gurobi is weird, but make sure it is downloaded and works first by getting it from the Gurobi website and obtaining an academic license. 

IF YOU PLAN ON SAVING DATA: go to folder 3. Functions/Functions.jl and modify the save_data command working directory use $pwd()$ command to help
^^^^^^^ Don't think this is a problem anymore because I have automated it but return later if problem. 


In [1]:
#Run this if setting up Julia on a new computer
#This will download all the packages needed to run the files
#Note: takes a few minutes
import Pkg
Pkg.add("Optim")
Pkg.add("Plots")
Pkg.add("StatsBase")
Pkg.add("Distributions")
Pkg.add("Statistics")
Pkg.add("Random")
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("PrettyTables")
Pkg.add("Distributed")
Pkg.add("TickTock")
Pkg.add("GLPK")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JLD2")
Pkg.add("Dates")
Pkg.add("FileIO")

Description of Structure of Code (As of November 11, 2021)

Everything to run is currently based off of a few main files. Look for the experiment you want to run:

1. Mulitiple runs at once among varying ADI and CV2 with similar demand totals 
     1. 1.Tradeoff Curves/1. Run Scenarios/ANY.ipynb file
    
Here you can build different networks to test while also varying ADI, CV2, Avg Dem, LT, AM Capacity, Network


2. Edit the code and functions above
    1.  1.Tradeoff Curves/2. Scripts/Modified_OptPolicy.ipynb

This notebook is where the model process is run
That notebook first uses generate_forecast.jl file to (based on a set of inputs described earlier, and cost inputs)
    
        1. Create Historical Demand
        2. Forecast future based on historical demand
        3. Find Markov Bootstrap of forecast to establish little S policy
        4. Create n number of actual demand realizations for the future. 
    
Then, We run the first MILP to find the optimal inventory policy which invovles solving for big S.

After all that we can evaluate each time period by time period and make decisions on orders and daily operations eventually getting the correct number of backorders and cost. 

Then we save in the folder 9. Raw Output Data. 

Best practices for loading data includes using and modifything the following function. As long as the naming convention is consistent and you set it up right, this is faster than changing the varying things by hand. It also depends on what exactly you want to load so you have to think about strucutures but typically when it is the example of average backorders, a single data file will have a single column of length networks you started. Then I hcat (horizontal concatenate) the varying files to get it into a single data frame. 


    function load_many_files(adi, cv2, folderpath, demlevel, starterstring, LT = "")    
        adistr = replace(string(adi[1]),"." => "_")
        cv2str = replace(string(cv2[1]),"." => "_")
        path = folderpath * starterstring[1] * LT[1] * "_" * adistr * "ADI_" * cv2str * "CV2_" * string(demlevel) *"AvgDem.jld2"
        file = FileIO.load(path)
        BO = file["BOtot"][:,1,1,1,1]
        TC = file["TC"]
        little_s = file["little_s"]
        for i = 2:length(adi)
            adistr = replace(string(adi[i]),"." => "_")
            cv2str = replace(string(cv2[i]),"." => "_")
            path = folderpath * starterstring[i] * LT[i] * "_" * adistr * "ADI_" * cv2str * "CV2_" * string(demlevel) *"AvgDem.jld2"
            file = FileIO.load(path)
            BO = hcat(BO, file["BOtot"][:,1,1,1,1])
            TC = hcat(TC,file["TC"][:,1,1])
            little_s = hcat(little_s,file["little_s"])

        end
        return BO, TC, little_s
    end

This section will detail more in depth pertinent files and what they do. 

